## Build Workflow from Fireworks yaml files

In [1]:
import yaml

In [2]:
from glob import glob

In [3]:
from fireworks import Firework, Workflow

In [4]:
from tabulate import tabulate

In [5]:
from pprint import PrettyPrinter

In [6]:
pp = PrettyPrinter(indent=2)

In [7]:
fws_yaml = sorted(glob("fw_*.yaml"))

In [8]:
print(tabulate( [ [ row ] for row in fws_yaml ] ,headers=["Files"],tablefmt='simple'))

Files
-----------------------------------------
fw_010_file_retrieval.yaml
fw_020_minimization_fixed_box.yaml
fw_030_minimization_relaxed_box.yaml
fw_040_minimization_slab_fixed_box.yaml
fw_050_minimization_slab_relaxed_box.yaml
fw_060_equilibration_nvt_slab.yaml
fw_070_equilibration_npt_slab.yaml


In [9]:
with open('dependencies.yaml') as stream:
    dependencies = yaml.safe_load(stream)

In [10]:
with open('metadata.yaml') as stream:
    metadata = yaml.safe_load(stream)

In [11]:
# identify subset of interlinked fw:

In [12]:
fws_set = set()

In [13]:
for k,v in dependencies.items():
    fws_set.update([k,*v])

In [14]:
fws_set

{'fw_010_file_retrieval.yaml',
 'fw_020_minimization_fixed_box.yaml',
 'fw_030_minimization_relaxed_box.yaml',
 'fw_040_minimization_slab_fixed_box.yaml',
 'fw_050_minimization_slab_relaxed_box.yaml',
 'fw_060_equilibration_nvt_slab.yaml',
 'fw_070_equilibration_npt_slab.yaml'}

In [15]:
pp.pprint(metadata)

{ 'metadata': { 'pbc': 111,
                'pressure': 0,
                'pressure_unit': 'atm',
                'sb_area': 2.20374e-16,
                'sb_area_unit': 'm^2',
                'sb_crystal_plane': 111,
                'sb_measures': [1.47e-08, 1.5e-08, 5.6e-09],
                'sb_measures_unit': 'm',
                'sb_multiples': [51, 30, 8],
                'sb_name': 'AU_111_51x30x8',
                'sb_normal': 2,
                'sb_thickness': 5.6e-09,
                'sb_thickness_unit': 'm',
                'sb_unit_cell': ['3e-10', '5e-10', '7e-10'],
                'sb_unit_cell_unit': 'm',
                'sb_volume': 3.11331e-25,
                'sb_volume_unit': 'm^3',
                'state': 'trial minimization with eam potential',
                'substrate': 'AU',
                'system_name': 'AU_111_51x30x8',
                'temperature': 298,
                'temperature_unit': 'K'},
  'name': 'NEMO TRIAL: LAMMPS minimization and equilibration

In [16]:
assert "metadata" in metadata

In [17]:
assert "name" in metadata

In [18]:
fws_id = { fw: (-10*(i+1)) for i, fw in enumerate(fws_set) }

In [19]:
pp.pprint(fws_id)

{ 'fw_010_file_retrieval.yaml': -30,
  'fw_020_minimization_fixed_box.yaml': -60,
  'fw_030_minimization_relaxed_box.yaml': -10,
  'fw_040_minimization_slab_fixed_box.yaml': -70,
  'fw_050_minimization_slab_relaxed_box.yaml': -20,
  'fw_060_equilibration_nvt_slab.yaml': -40,
  'fw_070_equilibration_npt_slab.yaml': -50}


In [20]:
fws_dict = { fw: Firework.from_file(fw, f_format='yaml') for fw in fws_set }

In [21]:
for name, fw in fws_dict.items():
    fw.fw_id = fws_id[name]

In [22]:
links = { 
    fws_dict[parent]: [ 
        fws_dict[child] for child in children ] for parent, children in dependencies.items() }

In [23]:
fws_list = list(fws_dict.values())

In [24]:
wf = Workflow( fws_list, links, name=metadata["name"], metadata=metadata["metadata"]  )

In [25]:
wf.to_file("wf.yaml",f_format='yaml')